In [ ]:
#import libraries
import sys
import os
import time
import datetime as dt
import numpy as np
import pandas as pd
import norgatedata as ng
from functions import trend_filter, position_size_atr, signal_filter, select_positions_1, \
                      exposure_summary, process_universe_factors, filter_ranks, step_activation, \
                      construct_trade_df
from ib_connect import main as positions

sys.path.append(os.path.abspath('../fin_data'))
from trade_logic.indicators import stock_beta, stock_volatility, atr, ewma
from utils.date_functions import last_business_day, time_elapsed
from utils.postgresql_data_query import get_bars, get_bm_bars, get_effective_dates
from data_updater.norgate_data import NorgateData

start_time = time.time()

In [ ]:
## set up parameters
bm_col = '$SPX'
lo_rank = 50
hi_rank = 50
w_offset = 1
d_offset = max(1, w_offset * 7)
beta_n = 36
vola_n = 20
atr_n = 20
ewma_stop = 128
ewma_n = 128
dgt = 2
cols = ['Date','Open','High','Low','Close']
factor_cols = ['ticker','factor_date','factor_definition_id','rank']
model_factors = [85, 86, 107, 7]
activation_factor = 85
activation_threshold = 90
activation_mult = 2
risk_factor = 0.001
sector_max = 0.20
max_positions = 200

excluded_tickers = ['VZIO']

client_id = 1
account_number = "DU3208934"
path = 'C:/Users/VadimKovshov/Dropbox/PYTHON/CSV_Data/momentum_model/'

use_postgresql = True
end = last_business_day(offset=d_offset)
start = last_business_day(offset=d_offset + 400)
current_date = dt.date.today()
print(f'Historical range: {end.strftime("%Y-%m-%d")} to {start.strftime("%Y-%m-%d")}')

In [ ]:
# get most recent positions from IB
positions, account_value = positions(client_id, account_number, path)

if positions.empty:
    print('No current positions found')
else:
    print(f'Current positions: {len(positions)}')
    positions.head(3)

if account_value:
    print(f'Account value: {account_value}')
else:
    account_value = 591912.26
    print(f'Using nominal account value: {account_value}')

position_tickers = positions['ticker'].tolist()
positions.head(3)

In [ ]:
# get universe factors for a given week
eff_date, pr_date = get_effective_dates(offset_0=w_offset)
print(f'Factor dates now: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')

ranks_now = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                     hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols)

positions_now = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                     hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols, tickers=position_tickers)

ranks_now = pd.concat([ranks_now, positions_now])
ranks_now.drop_duplicates(subset=['ticker'], inplace=True)

# get universe factors for one week prior
eff_date, pr_date = get_effective_dates(offset_0=w_offset+1)
print(f'Factor dates prior: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')

ranks_prior = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                       hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols)

positions_prior = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                       hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols, tickers=position_tickers)

ranks_prior = pd.concat([ranks_prior, positions_prior])
ranks_prior.drop_duplicates(subset=['ticker'], inplace=True)

In [ ]:
# Filter the DataFrames
filter_used = 1
df_now = filter_ranks(ranks_now, filter=filter_used)
df_prior = filter_ranks(ranks_prior, filter=filter_used)
combined_df = pd.merge(df_now, df_prior, on='ticker', how='left', suffixes=('', '_prior'), indicator=True)
# Assign labels based on the merge result
combined_df['label'] = combined_df['_merge'].map({'left_only': 'new', 'both': 'old'})

# Check if ticker is in position_tickers, and assign 'position' label
combined_df['label'] = np.where(combined_df['ticker'].isin(position_tickers), 'position', combined_df['label'])

combined_df['activation'] = combined_df.apply(
    lambda row: step_activation(
        row[f'f_{activation_factor}'], 
        row[f'f_{activation_factor}_prior'] if pd.notnull(row[f'f_{activation_factor}_prior']) else 0, 
        extreme_threshold=activation_threshold, 
        activation_mult=activation_mult
    ), axis=1
)

combined_df = combined_df.drop(columns=['_merge'])
columns_to_keep = list(df_now.columns) + ['label', 'activation']
combined_df = combined_df[columns_to_keep]

# Optionally, reset the index
combined_df = combined_df.reset_index(drop=True)

# Filter tickers and limit to max_positions
tickers = (
    combined_df
    .loc[~combined_df['ticker'].isin(excluded_tickers), 'ticker']
    .unique()
    .tolist()[:max_positions]
)

print(f'Filtered tickers: {len(tickers)}')

In [ ]:
# get industry data
ng_obj = NorgateData(symbols= tickers, database='US Stocks')
ng_obj.process_metadata()
industry_df = ng_obj.df_result['metadata'].rename(columns={'symbol':'ticker', 'security_name':'name', 'GICS_level_1': 'sector', 'GICS_level_2': 'industry_group',
                                                            'GICS_level_3': 'industry', 'GICS_level_4': 'sub_industry'}).reset_index(drop=True)
industry_df = industry_df[['ticker', 'sector', 'industry_group', 'industry', 'sub_industry', 'name']]

sorted_df = pd.merge(combined_df, industry_df, on='ticker', how='left')

print (f'Sector tickers: {len(tickers)}')

In [ ]:
# get benchmark data
if use_postgresql:
    bm_df = get_bm_bars(bm_col,start,end) # uses saved PostgreSQL data
else:
    bm_df = ng_obj.daily_bars('$SPX', start, end, interval='D') # uses NorgateData data
    bm_df = bm_df[cols]
    bm_df['Ticker'] = bm_col

print(f'Benchmark data: {len(bm_df)}')

In [ ]:
exposure = trend_filter(bm_df, _c='Close')
exposure = exposure[-1]
print(f'Exposure: {exposure}')

In [ ]:
# get stock data and process signals
df_f = pd.DataFrame()
i = 0
for t in tickers:
    i += 1
    try:
        if use_postgresql:
            st_df = get_bars([t],start,end,backtest=False) # uses saved PostgreSQL data
        else:
            st_df = ng_obj.daily_bars(t, start, end, interval='D') # uses NorgateData data
            st_df = st_df[cols]
            st_df['Ticker'] = t

        beta_value = round(stock_beta(df=st_df, _c='Close', bm_df=bm_df, bm='Close', n=beta_n),dgt)
        vola_value = stock_volatility(df=st_df, _c='Close', n=vola_n, annualized=False)
        atr_value = round(atr(df=st_df, _h='High', _l='Low', _c='Close', n=atr_n).iloc[-1],2)
        ewma_value = round(ewma(df=st_df, _c='Close', _n=ewma_stop, _mp=ewma_stop).iloc[-1],dgt)
        last_close = st_df['Close'].iloc[-1]
        activation_value = combined_df.loc[combined_df['ticker'] == t, 'activation'].values[0]
        signal_value = signal_filter(df=st_df, _h='High', _l='Low', _c='Close', n=ewma_n, pct_change_threshold=0.15, period=90)
        atr_position_size = activation_value * signal_value * position_size_atr(account_value, risk_factor, exposure, atr_value) // 10 * 10
        dollar_position = atr_position_size * st_df['Close'].iloc[-1]
        pct_position = round(dollar_position / account_value,dgt)

        data = {
            'date': st_df['Date'].iloc[-1],
            'ticker': t,
            'beta': beta_value,
            'volatility': vola_value,
            'atr': atr_value,
            'last_close': last_close,
            'ewma_stop': ewma_value,
            'signal': signal_value,
            'new_position': atr_position_size,
            'dollar_position': dollar_position,
            'pct_position': pct_position,
            'activation_value': activation_value
        }
        df = pd.DataFrame([data])
        df_f = pd.concat([df_f,df],ignore_index=True)
        if i % 20 == 0:
            print(f'Completed: {i}, {t}, {time_elapsed(start_time)}')
    except Exception as e:
        print(f"Error processing {t}: {e}")  # Print out the exception for debugging
        continue

print(f'Completed: {i}, {t}, {time_elapsed(start_time)}')

In [10]:
# final filter
df_f_final = pd.merge(sorted_df, df_f, on='ticker', how='left')
df_f_final = df_f_final[df_f_final['signal'] == 1].reset_index(drop=True).copy()
df_f_final = df_f_final.drop(columns=['factor_date']).reset_index(drop=True)

In [11]:
# create model output and trade dataframes
selected_positions_df = select_positions_1(df_f_final, max_exposure=exposure, max_sector_allocation=sector_max)
trade_df = construct_trade_df(positions, selected_positions_df)

In [ ]:
summary_positions = selected_positions_df[selected_positions_df['new_position'] != 0]
styled_summary = exposure_summary(summary_positions)

print(f'Completed: {time_elapsed(start_time)}')
styled_summary

In [ ]:
# save output to csv
selected_positions_df.to_csv(f'{path}momo_model_output_{current_date}.csv', index=False)
trade_df.to_csv(f'{path}momo_trade_file_{current_date}.csv', index=False)
trade_df